In [2]:
import boto3  # pip install boto3
#
# Let's use Amazon S3
s3 = boto3.resource("s3")

# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

clean-googleplay-bucket
jump-googleplay-bucket


In [3]:
import mysql.connector
import sys
import os
import pandas as pd
import sqlalchemy

In [4]:
s3 = boto3.client("s3")

s3.upload_file(
    Filename="./googleplaystore_user_reviews.csv", 
    Bucket="jump-googleplay-bucket",
    Key="googleplaystore_user_reviews.csv", 
)

In [5]:
s3.download_file(
    Bucket="jump-googleplay-bucket", Key="googleplaystore_user_reviews.csv", Filename="./googleplaystore_user_reviews.csv"
)
s3.download_file(
    Bucket="jump-googleplay-bucket", Key="googleplaystore.csv", Filename="./googleplaystore.csv"
)

In [6]:
ENDPOINT="google-play-database.cdhyacptpzwk.us-east-2.rds.amazonaws.com"
PORT="3306"
USER="root"
REGION="us-east-2"
DBNAME="GooglePlay"
os.environ['LIBMYSQL_ENABLE_CLEARTEXT_PLUGIN'] = '1'

#gets the credentials from .aws/credentials
session = boto3.Session(profile_name='default')
client = session.client('rds')

#token = client.generate_db_auth_token(DBHostname=ENDPOINT, Port=PORT, DBUsername=USER, Region=REGION)   

try:
    conn =  mysql.connector.connect(host=ENDPOINT, user=USER, passwd='rootroot', port=PORT, database=DBNAME)
    cur = conn.cursor()
    cur.execute("""SELECT now()""")
    query_results = cur.fetchall()
    print(query_results)
except Exception as e:
    print("Database connection failed due to {}".format(e))         

[(datetime.datetime(2022, 10, 5, 15, 11, 59),)]


In [7]:
cur.execute('CREATE DATABASE GooglePlay;')

DatabaseError: 1007 (HY000): Can't create database 'GooglePlay'; database exists

In [8]:
playstore_df = pd.read_csv('./googleplaystore.csv')
reviews_df = pd.read_csv('./googleplaystore_user_reviews.csv')
playstore_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [9]:
reviews_df.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [10]:
playstore_df = playstore_df.dropna()
reviews_df = reviews_df.dropna()

In [11]:
playstore_df.index.name='APP_ID'
reviews_df.index.name='APP_ID'

In [12]:
playstore_df.dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

In [13]:
reviews_df.dtypes

App                        object
Translated_Review          object
Sentiment                  object
Sentiment_Polarity        float64
Sentiment_Subjectivity    float64
dtype: object

In [14]:
playstore_df['Last Updated'] = pd.to_datetime(playstore_df['Last Updated'])
playstore_df['Reviews'] = playstore_df['Reviews'].astype(int)
playstore_df.dtypes

App                       object
Category                  object
Rating                   float64
Reviews                    int32
Size                      object
Installs                  object
Type                      object
Price                     object
Content Rating            object
Genres                    object
Last Updated      datetime64[ns]
Current Ver               object
Android Ver               object
dtype: object

In [15]:
playstore_df[playstore_df['Type'] != "Free"] 

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
APP_ID,,,,,,,,,,,,,
234,TurboScan: scan documents and receipts in PDF,BUSINESS,4.7,11442,6.8M,"100,000+",Paid,$4.99,Everyone,Business,2018-03-25,1.5.2,4.0 and up
235,Tiny Scanner Pro: PDF Doc Scan,BUSINESS,4.8,10295,39M,"100,000+",Paid,$4.99,Everyone,Business,2017-04-11,3.4.6,3.0 and up
290,TurboScan: scan documents and receipts in PDF,BUSINESS,4.7,11442,6.8M,"100,000+",Paid,$4.99,Everyone,Business,2018-03-25,1.5.2,4.0 and up
291,Tiny Scanner Pro: PDF Doc Scan,BUSINESS,4.8,10295,39M,"100,000+",Paid,$4.99,Everyone,Business,2017-04-11,3.4.6,3.0 and up
427,Puffin Browser Pro,COMMUNICATION,4.0,18247,Varies with device,"100,000+",Paid,$3.99,Everyone,Communication,2018-07-05,7.5.3.20547,4.1 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10690,FO Bixby,PERSONALIZATION,5.0,5,861k,100+,Paid,$0.99,Everyone,Personalization,2018-04-25,0.2,7.0 and up
10697,Mu.F.O.,GAME,5.0,2,16M,1+,Paid,$0.99,Everyone,Arcade,2017-03-03,1.0,2.3 and up
10760,Fast Tract Diet,HEALTH_AND_FITNESS,4.4,35,2.4M,"1,000+",Paid,$7.99,Everyone,Health & Fitness,2018-08-08,1.9.3,4.2 and up


In [16]:
playstore_df['Price'] = playstore_df['Price'].str.replace('$','').astype(float)

C:\Users\Harsh\AppData\Local\Temp\ipykernel_28832\568852492.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  playstore_df['Price'] = playstore_df['Price'].str.replace('$','').astype(float)


In [18]:
playstore_df['Installs'] = playstore_df['Installs'].str.replace('+','').str.replace(',','').astype(int)

C:\Users\Harsh\AppData\Local\Temp\ipykernel_28832\435226539.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  playstore_df['Installs'] = playstore_df['Installs'].str.replace('+','').str.replace(',','').astype(int)


In [29]:
print(playstore_df['Size'].unique())

['19M' '14M' '8.7M' '25M' '2.8M' '5.6M' '29M' '33M' '3.1M' '28M' '12M'
 '20M' '21M' '37M' '5.5M' '17M' '39M' '31M' '4.2M' '23M' '6.0M' '6.1M'
 '4.6M' '9.2M' '5.2M' '11M' '24M' '9.4M' '15M' '10M' '1.2M' '26M' '8.0M'
 '7.9M' '56M' '57M' '35M' '54M' '201k' '3.6M' '5.7M' '8.6M' '2.4M' '27M'
 '2.7M' '2.5M' '7.0M' '16M' '3.4M' '8.9M' '3.9M' '2.9M' '38M' '32M' '5.4M'
 '18M' '1.1M' '2.2M' '4.5M' '9.8M' '52M' '9.0M' '6.7M' '30M' '2.6M' '7.1M'
 '22M' '6.4M' '3.2M' '8.2M' '4.9M' '9.5M' '5.0M' '5.9M' '13M' '73M' '6.8M'
 '3.5M' '4.0M' '2.3M' '2.1M' '42M' '9.1M' '55M' '23k' '7.3M' '6.5M' '1.5M'
 '7.5M' '51M' '41M' '48M' '8.5M' '46M' '8.3M' '4.3M' '4.7M' '3.3M' '40M'
 '7.8M' '8.8M' '6.6M' '5.1M' '61M' '66M' '79k' '8.4M' '3.7M' '118k' '44M'
 '695k' '1.6M' '6.2M' '53M' '1.4M' '3.0M' '7.2M' '5.8M' '3.8M' '9.6M'
 '45M' '63M' '49M' '77M' '4.4M' '70M' '9.3M' '8.1M' '36M' '6.9M' '7.4M'
 '84M' '97M' '2.0M' '1.9M' '1.8M' '5.3M' '47M' '556k' '526k' '76M' '7.6M'
 '59M' '9.7M' '78M' '72M' '43M' '7.7M' '6.3M' '33

In [28]:
playstore_df = playstore_df[playstore_df['Size'] != 'Varies with device']

In [40]:
playstore_df['size_suffix'] = playstore_df['Size'].str.get(-1)

C:\Users\Harsh\AppData\Local\Temp\ipykernel_28832\1588734516.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playstore_df['size_suffix'] = playstore_df['Size'].str.get(-1)


In [44]:
playstore_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,size_suffix
APP_ID,,,,,,,,,,,,,,
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,10000,Free,0.0,Everyone,Art & Design,2018-01-07,1.0.0,4.0.3 and up,M
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,2018-01-15,2.0.0,4.0.3 and up,M
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,5000000,Free,0.0,Everyone,Art & Design,2018-08-01,1.2.4,4.0.3 and up,M
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,50000000,Free,0.0,Teen,Art & Design,2018-06-08,Varies with device,4.2 and up,M
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,100000,Free,0.0,Everyone,Art & Design;Creativity,2018-06-20,1.1,4.4 and up,M


In [46]:
playstore_df['Size'] = playstore_df['Size'].str.replace('M','')
playstore_df['Size'] = playstore_df['Size'].str.replace('k','')

C:\Users\Harsh\AppData\Local\Temp\ipykernel_28832\2245014916.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playstore_df['Size'] = playstore_df['Size'].str.replace('M','')
C:\Users\Harsh\AppData\Local\Temp\ipykernel_28832\2245014916.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playstore_df['Size'] = playstore_df['Size'].str.replace('k','')


In [48]:
playstore_df['Size'] = playstore_df['Size'].astype(float)

C:\Users\Harsh\AppData\Local\Temp\ipykernel_28832\1779049080.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playstore_df['Size'] = playstore_df['Size'].astype(float)


In [49]:
playstore_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,size_suffix
APP_ID,,,,,,,,,,,,,,
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,Free,0.0,Everyone,Art & Design,2018-01-07,1.0.0,4.0.3 and up,M
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,2018-01-15,2.0.0,4.0.3 and up,M
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,Free,0.0,Everyone,Art & Design,2018-08-01,1.2.4,4.0.3 and up,M
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,Free,0.0,Teen,Art & Design,2018-06-08,Varies with device,4.2 and up,M
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,Free,0.0,Everyone,Art & Design;Creativity,2018-06-20,1.1,4.4 and up,M


In [57]:
playstore_df['Size'] = playstore_df[playstore_df['size_suffix'] == "M"]['Size'] * 1000

C:\Users\Harsh\AppData\Local\Temp\ipykernel_28832\1505575508.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playstore_df['Size'] = playstore_df[playstore_df['size_suffix'] == "M"]['Size'] * 1000


In [58]:
playstore_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,size_suffix
APP_ID,,,,,,,,,,,,,,
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000.0,10000,Free,0.0,Everyone,Art & Design,2018-01-07,1.0.0,4.0.3 and up,M
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,2018-01-15,2.0.0,4.0.3 and up,M
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700.0,5000000,Free,0.0,Everyone,Art & Design,2018-08-01,1.2.4,4.0.3 and up,M
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25000.0,50000000,Free,0.0,Teen,Art & Design,2018-06-08,Varies with device,4.2 and up,M
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2800.0,100000,Free,0.0,Everyone,Art & Design;Creativity,2018-06-20,1.1,4.4 and up,M
